# Section 1.2: Dimension reduction and principal component analysis (PCA)

One of the iron laws of data science is know as the "[curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality)": as the number of considered features (dimensions) of a space increases, the number of data configurations grows exponentially. Thus the number of observations (data points) needed to account for these configurations must also increase. This fact of life has huge ramifications for the time, computational effort, and memory required. As a result, it is often desirable to reduce the number of dimensions we work with.

The challenge is determining which features we can remove from our analysis.

Not all features are created equal. Certain features may impede our analysis, or simply not be overly important. We need to identify the features to exclude, but obviously we can only take this so far. At a certain point reducing dimensions will also reduce the accuracy of a classifier and our analysis.

## Our scenario

We have a collection of foods and their nutrients. We'd like to group them together so we can see similar items. Our traditional categorization of dairy or vegetables aren't descriptive enouth, and can be a little misleading. For example, not all fruits are created equal - an avocado is rather different from a apple. We'd like to group our foods together based on their composition.

As we begin our analysis, our initial challenge is trying to determine what makes a food item a food item? What components are most important? It's a real struggle to try to determine what components we should focus on.

## PCA in theory

One way to reduce the number of dimensions we have to work with is by projecting (or converting) our feature space into a lower dimensional space. The reason why we can do this is that in most real-world problems, data points are not spread uniformly across all dimensions. Some features might be near constant, while others are highly correlated, which means that those data points lie close to a lower-dimensional subspace.


### An analogy

We can picture the process of PCA reducing the number of dimensions as we would, well, a picture. A picture is a two-dimensional representation of a three-dimensional world. But the reason a pixel sits where it does on a picture and looks how it looks is because of where it is on the X, Y and Z coordinates which make up the world we see. The picture isn't merely taking a two-dimensional slice, but rather consolidating three dimensions into two **new** dimensions.

This is especially evident in a picture with a [bokeh](https://en.wikipedia.org/wiki/Bokeh) effect (those pictures where the subject is in focus and everything else is blurry). The photographer has exerted a fair bit of control over how the conversion from 3-D to 2-D is going to take place. The pixels making up our subject will show as sharp, while all other pixels will be fuzzy. Basically, the photographer is drawing our eyes to what's important.

We can do the same with the dimensions (or columns) in our data, reducing the number and allowing us to focus (pun intended) on the important information.

## PCA in action

PCA behaves much in the same way, by allowing us to focus on what's important, specifically variance. Where are we seeing a spread of data? Where are we seeing change? What's truly impacting the data?

Let's take a look at an example of a three dimensional data space as represented in the image below. The data points are not spread across the entire plane, but are nicely clumped, roughly in an oval. Because the cluster (or, indeed, any cluster) is roughly elliptical, it can be mathematically described by two values: its major (long) axis and its minor (short) axis. These axes form the *principal components* of the cluster. Basically, this is where the magic is happening for our data.

<img align="center" style="padding-right:10px;" src="Images/PCA.png">

We can **construct a whole new feature space** around this cluster, defined by two *eigenvectors* (the vectors that define the linear transformation to this new feature space), $c_{1}$ and $c_{2}$. Better still, we don't have to consider all of the dimensions of this new space. Intuitively, we can see that most of the points lie on or close to the line that runs through $c_{1}$. So, if we project the cluster down from two dimensions to that single dimension, we capture most of the information about this data sense while simplifying our analysis. This ability to extract most of the information from a dataset by considering only a fraction of its definitive eigenvectors forms the heart of principal component analysis (PCA).

## Import modules and dataset

You will need to clean and prepare the data in order to conduct PCA on it, so pandas will be essential. You will also need NumPy, a bit of Scikit Learn, and pyplot.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

The dataset we’ll use here is the same one drawn from the [U.S. Department of Agriculture National Nutrient Database for Standard Reference](https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/nutrient-data-laboratory/docs/usda-national-nutrient-database-for-standard-reference/) that you prepared in Section 1.1. Remember to set the encoding to `latin_1` (for those darn µg).

In [ ]:
df = pd.read_csv('Data/USDA-nndb-combined.csv', encoding='latin_1')

We can check the number of columns and rows using the `info()` method for the `DataFrame`.

In [ ]:
df.info()

> **Exercise**
>
> Can you think of a more concise way to check the number of rows and columns in a `DataFrame`? (***Hint:*** Use one of the [attributes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) of the `DataFrame`.)

## Handle `null` values

Because this is a real-world dataset, it is a safe bet that it has `null` values in it. We could first check to see if this is true. However, later on in this section, we will have to transform our data using a function that cannot use `NaN` values, so we might as well drop rows containing those values.

> **Exercise**
>
> Drop rows from the `DataFrame` that contain `NaN` values. (If you need help remembering which method to use, see [this page](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).)

In [ ]:
# Run the code to drop all null and NaN values


Now let’s see how many rows we have left.

In [ ]:
df.shape

Dropping those rows eliminated 76 percent of our data (8989 entries to 2190). An imperfect state of affairs, but we still have enough for our purposes in this section.

> **Key takeaway:** Another solution to removing `null` values is to impute values for them, but this can be tricky. Should we handle missing values as equal to 0? What about a fatty food with `NaN` for `Lipid_Tot_(g)`? We could try taking the averages of values surrounding a `NaN`, but what about foods that are right next to rows containing foods from radically different food groups? It is possible to make justifiable imputations for missing values, but it can be important to involve subject-matter experts (SMEs) in that process.

## Split off descriptive columns

Our descriptive columns (such as `FoodGroup` and `Shrt_Desc`) pose challenges for us when it comes time to perform PCA because they are categorical rather than numerical features, so we will split our `DataFrame` in to one containing the descriptive information and one containing the nutritional information.

In [ ]:
desc_df = df.iloc[:, [0, 1, 2]+[i for i in range(50,54)]]
desc_df.set_index('NDB_No', inplace=True)
desc_df.head()

> **Question**
>
> Why was it necessary to structure the `iloc` method call the way we did in the code cell above? What did it accomplish? Why was it necessary set the `desc_df` index to `NDB_No`?

In [ ]:
nutr_df = df.iloc[:, :-5] # Remove the last five columns
nutr_df.head()

> **Question**
>
> What did the `iloc` syntax do in the code cell above?

In [ ]:
nutr_df = nutr_df.drop(['FoodGroup', 'Shrt_Desc'], axis=1)

> **Exercise**
>
> Now set the index of `nutr_df` to use `NDB_No`.

> **Exercise solution**
>
> The correct code for students to use here is `nutr_df.set_index('NDB_No', inplace=True)`.

Now let’s take a look at `nutr_df`.

In [ ]:
nutr_df.head()

## Normalize and center the data

Our numeric data comes in a variety of mass units (grams, milligrams, and micrograms) and one energy unit (kilocalories). In order to make an apples-to-apples comparison (pun intended) of the nutritional data, we need to first *normalize* the data and make it more normally distributed (that is, make the distribution of the data look more like a familiar bell curve). PCA needs relatively normalized data to perform its magic.

To help see why we need to normalize the data, let's look at a histogram of all of the columns.

In [ ]:
ax = nutr_df.hist(bins=50, xlabelsize=-1, ylabelsize=-1, figsize=(11,11))

### Our first transformation - Box-Cox

Not a bell curve in sight. Worse, a lot of the data is clumped at or around 0. There are a number of ways we can massage the data into a a more useable state.

We're going to start by trying the [Box-Cox Transformation](https://www.statisticshowto.datasciencecentral.com/box-cox-transformation/) on the data, a popular transformation. It does require a strictly positive input, so we will add 1 to every value in each column.

In [ ]:
nutr_df = nutr_df + 1

Now for the transformation. The Box-Cox Transformation performs the transformation $y(\lambda) = \dfrac{y^{\lambda}-1}{\lambda}$ for $\lambda \neq 0$ and $y(\lambda) = log y$ for $\lambda = 0$ for all values $y$ in a given column. SciPy has a particularly useful `boxcox()` function that can automatically calculate the $\lambda$ for each column that best normalizes the data in that column. (However, it is does not support `NaN` values; scikit-learn has a comparable `boxcox()` function that is `NaN`-safe, but it is not available on the version of scikit-learn that comes with Azure notebooks.)

In [ ]:
from scipy.stats import boxcox

nutr_df_TF = pd.DataFrame(index=nutr_df.index)
for col in nutr_df.columns.values:
    nutr_df_TF['{}_TF'.format(col)] = boxcox(nutr_df.loc[:, col])[0]

Let's now take a look at the `DataFrame` containing the transformed data.

In [ ]:
ax = nutr_df_TF.hist(bins=50, xlabelsize=-1, ylabelsize=-1, figsize=(11,11))

A few of the columns now look about right, but unfortunately we really didn't see the results we were hoping for, which was bell curves for the bulk of our columns.

### Standard scaling

Our data units were incompatible to begin with, and the transformations haven't given us any real improvement. We can apply a different technique - centering the data around 0; that is, we will again transform the data, this time so that every column has a mean of 0 and a standard deviation of 1. Put another way, we're going to standardize our data around 0. Scikit-learn has a convenient function for this - [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). We're going to call `fit_transform` to perform the conversion. Please note `fit_transform` will return a `numpy` array rather than a `DataFrame`.

In [ ]:
nutr_df_TF = StandardScaler().fit_transform(nutr_df_TF)

You can satisfy your self that the data is now centered by using the `mean()` method on the array.

In [ ]:
print("mean: ", np.round(nutr_df_TF.mean(), 2))

> **Exercise**
>
> Find the standard deviation for the `nutr_df_TF`. (If you need a hint, you can always use **tab** in Jupyter notebooks)

## PCA in practice

It is finally time to perform the PCA on our data. (As stated before, even with pretty clean data, a lot of effort has to go into preparing the data for analysis.)

In [ ]:
fit = PCA()
pca = fit.fit_transform(nutr_df_TF)

So, now that we have peformed the PCA on our data, what do we actually have? Remember that PCA is foremost about finding the eigenvectors for our data. We then want to select some subset of those vectors to form the lower-dimensional subspace in which to analyze our data.

Let's take a look at a scatter plot created by the first two components. We can already see some clustering, which we'll be able to better represent when we get towards the end.

In [ ]:
plt.scatter(pca[:,0], pca[:,1], alpha='.1')
plt.xlabel('C1')
plt.ylabel('C2')
plt.title('Collapsed data')

Not all of the eigenvectors are created equal. Just a few of them will account for the majority of the variance in the data. (Put another way, a subspace composed of just a few of the eigenvectors will retain the majority of the information from our data.) We want to focus on those vectors.

To help us get a sense of how many vectors we should use, consider this scree graph of the variance for the PCA components, which plots the variance explained by the components from greatest to least.

In [ ]:
plt.plot(fit.explained_variance_ratio_)

This is where data science can become an art. As a rule of thumb, we want to look for "elbow" in the graph, which is the point at which the few components have captured the majority of the variance in the data (after that point, we are only adding complexity to the analysis for increasingly diminishing returns). In this particular case, that appears to be at about five components.

We can take the cumulative sum of the first five components to see how much variance they capture in total.

In [ ]:
print(fit.explained_variance_ratio_[:5].sum())

So our five components capture about 70 percent of the variance. We can see what fewer or additional components would yield by looking at the cumulative variance for all of the components. You can see increasing to 6 components would cover an additional 3 percentage points over the first 5, and there's an ever-diminisioning return.

In [ ]:
fit.explained_variance_ratio_.cumsum()

We can also examine this visually.

In [ ]:
plt.plot(np.cumsum(fit.explained_variance_ratio_))
plt.title("Cumulative Explained Variance Graph")

Ultimately, it is a matter of judgment as to how many components to use, but five vectors (and 70 percent of the variance) will suffice for our purposes in this section.

To aid further analysis, let's now put those five components into a DataFrame.

In [ ]:
pca_df = pd.DataFrame(pca[:, :5], index=desc_df.index)
pca_df.head()

Each column represents one of the eigenvectors, and each row is one of the coordinates that defines that vector in five-dimensional space.

We will want to add the FoodGroup column back in to aid with our interpretation of the data later on. Let's also rename the component-columns $c_{1}$ through $c_{5}$ so that we know what we are looking at.

In [ ]:
pca_df = pca_df.join(desc_df)
pca_df.drop(['FoodGroup', 'GmWt_Desc1', 'GmWt_2', 'GmWt_Desc2', 'Refuse_Pct', 'Shrt_Desc'], axis=1, inplace=True)
pca_df.rename(columns={0:'c1', 1:'c2', 2:'c3', 3:'c4', 4:'c5'}, inplace=True)

Don't worry that the FoodGroup column has all `NaN` values: it is not a vector, so it has no vector coordinates.

One last thing we should demonstrate is that each of the components is mutually perpendicular (or orthogonal in math-speak). One way of expressing that condition is that each component-vector should perfectly correspond with itself and not correlate at all (positively or negatively) with any other vector.

In [ ]:
np.round(pca_df.corr(), 5)

## Interpreting the results

What do our new vectors mean? What is driving our data? To see these results, we will create pandas Series for each of the components, index them by feature, and then sort them in descreasing order (so that a higher number represents a feature that is positively correlated with that vector and negative numbers represent low correlation).

In [ ]:
vects = fit.components_[:5]

In [ ]:
c1 = pd.Series(vects[0], index=nutr_df.columns)
c1.sort_values(ascending=False)

Our first component is primarily driven by the amounts of protein, selenium and zinc, while the amounts of vitamin C and sugar have a lessened impact.

In [ ]:
c2 = pd.Series(vects[1], index=nutr_df.columns)
c2.sort_values(ascending=False)

Our second component is primarily driven by the amounts of fiber, manganese and folate.

By using PCA we've been able to distill our data down into fewer, more meaningful dimensions and gain insights we might miss otherwise.

## Using our PCA results for categorization

One of the most common uses of PCA is to prepare data for categorization. PCA provides meaningful dimensions which are non-correlated, so our cluster will typically provide better results. This is why we started with PCA to before we attempted to cluster our foods together.

To perform the categorization, we're going to use [k-means clustering](https://en.wikipedia.org/wiki/K-means_clustering). In a nutshell, k-means will look to identify groups (or clusters) in our data.

We're going to start by loading the library, creating our model, and then fitting our PCA results into the model. We are arbitrarily using 3 clusters; in [Machine Learning 2](https://github.com/microsoft/Reactors/tree/main/Machine_Learning_2) there is a deeper conversation of k-means and how to determine the best number of clusters. We're using just the first 5 components as we determined it will provide the best return on investment.

In [ ]:
from sklearn.cluster import KMeans
kmeansmodel = KMeans(n_clusters=3, random_state=13)
kmeansmodel.fit(pca[:, :5])
kmeansmodel.labels_

Let's recreate our `DataFrame` with the PCA values, a new column named **Cluster** which will hold the `label_` or cluster number for each food item, and add back in the text descriptions.

In [ ]:
# Create a new data frame
pca_df = pd.DataFrame(pca[:, :5], index=desc_df.index)

# Add in the cluster number
pca_df['Cluster'] = kmeansmodel.labels_

# Add in the text descriptions
pca_df = pca_df.join(desc_df)
pca_df.head()

### Exploring our clusters

Let's see the products which wound up in the first cluster.

In [ ]:
pca_df.loc[pca_df['Cluster'] == 0]['Shrt_Desc'][:500]

> **Exercise** Display the first 500 rows for the remaining two clusters

In [ ]:
# Display cluster two (remember indexing starts at 0)


In [ ]:
# Display cluster three (remember indexing starts at 0)


### Conclusion

Reducing dimensions to allow us to focus on what's important can be a bit of a a challenge. And, frankly, PCA can be a bit confusing at first. PCA's main goal is to provide us with a better signal to noise ratio. It helps us weed out the dimensions which aren't as important, and provide more meaningful insights into the relationships between our data. It's commonly used before categorization algorithms, such as k-means.